# LAB 6: INSURANCE CLAIMS PROCESSING - CAPSTONE PROJECT

**Course:** Advanced Prompt Engineering Training  
**Session:** Session 6 - Advanced BFSI Applications (Day 3)  
**Duration:** 120 minutes (2 hours)  

---

## LEARNING OBJECTIVES

Build a **production-ready insurance claims processing system** integrating:

✓ RAG for policy knowledge retrieval  
✓ Orchestration for multi-step workflows  
✓ Agents for autonomous decision-making  
✓ RAGAs for quality evaluation  
✓ Compliance with audit trails  

---

## SETUP & INSTALLATION

In [ ]:
# Import all required libraries

import os
import json
import time
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional, Any
from dataclasses import dataclass, field
import random
import warnings
warnings.filterwarnings('ignore')

# LangChain
from langchain_classic.agents import create_react_agent, AgentExecutor, Tool
from langchain_classic.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_classic.schema import Document as LangChainDocument
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter

# Vector DB
import chromadb
from chromadb.config import Settings

from dotenv import load_dotenv

load_dotenv(override=True)

# RAGAs Evaluation
try:
    from ragas import evaluate
    from ragas.metrics import (
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    )
    from datasets import Dataset
    RAGAS_AVAILABLE = True
except ImportError:
    print("⚠️ RAGAs not available - evaluation will be skipped")
    RAGAS_AVAILABLE = False

# Utilities
import pandas as pd
import numpy as np

print("✓ All libraries imported successfully")

In [ ]:
# Initialize OpenAI client
from openai import OpenAI

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable is not set. Please set it to your OpenAI API key.")

client = OpenAI(api_key=api_key)

model_name = os.environ.get("MODEL_NAME", "gpt-4o")
fastmodel_name = os.environ.get("FAST_MODEL_NAME", "gpt-3.5-turbo")

# LangChain models
llm_gpt4 = ChatOpenAI(model=model_name, temperature=0)
llm_gpt35 = ChatOpenAI(model=fastmodel_name, temperature=0)

# Embedding model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

print("✓ Models configured: GPT-4, GPT-3.5, text-embedding-3-small")

---

## PART 1: DATA PREPARATION (15 minutes)

Creating synthetic insurance policies and claims data.

### 1.1 Insurance Policy Documents

In [ ]:
# Create comprehensive insurance policy documents

INSURANCE_POLICIES = {
    "auto_insurance_policy": """SECURELIFE AUTO INSURANCE POLICY
Policy Number: AUTO-2024-STANDARD

SECTION 1: COVERAGE OVERVIEW
1.1 Collision Coverage
- Maximum coverage: $50,000 per incident
- Deductible: $1,000
- Covers damage from collisions

1.2 Comprehensive Coverage  
- Maximum coverage: $50,000
- Deductible: $500
- Covers theft, vandalism, weather damage

1.3 Liability Coverage
- Bodily injury: $100,000 per person, $300,000 per accident
- Property damage: $50,000

1.4 Medical Payments
- Coverage: $10,000 per person

1.5 Towing and Rental
- Towing: $200 per incident  
- Rental: $50/day, max 30 days

SECTION 2: EXCLUSIONS
- Intentional damage
- Driving under influence
- Racing or competitive events
- Commercial use without commercial policy""",

    "home_insurance_policy": """SECURELIFE HOMEOWNERS INSURANCE POLICY
Policy Number: HOME-2024-PREMIUM

SECTION 1: DWELLING COVERAGE
1.1 Structure Coverage
- Coverage limit: $500,000
- Replacement cost coverage
- Deductible: $2,500

1.2 Personal Property
- Coverage: $350,000 (70% of dwelling)
- Replacement cost
- Deductible: $1,000

1.3 Liability Coverage
- Coverage: $500,000 per occurrence
- Legal defense included

1.4 Additional Living Expenses
- Coverage: $100,000 (20% of dwelling)
- Hotel, meals, temporary housing

SECTION 2: EXCLUSIONS
- Flood (separate policy required)
- Earthquake (separate policy)
- Normal wear and tear
- Intentional damage""",

    "health_insurance_policy": """SECURELIFE HEALTH INSURANCE POLICY
Policy Number: HEALTH-2024-GOLD

SECTION 1: IN-NETWORK BENEFITS
1.1 Deductibles and Coinsurance
- Annual deductible: $2,000 individual, $4,000 family
- Coinsurance: 80/20 (plan pays 80%)
- Out-of-pocket max: $6,000 individual

1.2 Hospital Services
- Emergency room: $500 copay, then 80% after deductible
- Urgent care: $75 copay
- Ambulance: 80% after deductible

1.3 Physician Services
- Primary care: $30 copay
- Specialist: $60 copay
- Preventive care: 100% covered

1.4 Prescription Drugs
- Generic: $10 copay
- Preferred brand: $40 copay
- Non-preferred: $70 copay

SECTION 2: EXCLUSIONS
- Cosmetic surgery (unless medically necessary)
- Experimental treatments
- Services outside USA (except emergencies)"""
}

# Create LangChain documents
policy_documents = []
for policy_name, content in INSURANCE_POLICIES.items():
    doc = LangChainDocument(
        page_content=content,
        metadata={
            "source": policy_name,
            "type": "insurance_policy",
            "created_at": datetime.now().isoformat()
        }
    )
    policy_documents.append(doc)

print(f"✓ Created {len(policy_documents)} insurance policy documents")
for doc in policy_documents:
    print(f"  - {doc.metadata['source']}: {len(doc.page_content)} characters")

### 1.2 Sample Insurance Claims

In [ ]:
# Create sample insurance claims for testing

SAMPLE_CLAIMS = [
    {
        "claim_id": "CLM-2024-001",
        "policy_number": "AUTO-2024-STANDARD",
        "claim_type": "auto",
        "date_of_incident": "2024-02-08",
        "date_reported": "2024-02-08",
        "customer_name": "John Smith",
        "description": """Single vehicle accident on I-95 during heavy rain.
        Hit guardrail. Front bumper damaged, headlight broken.
        Minor fender damage. No injuries. Police report filed.""",
        "estimated_damage": 3500,
        "supporting_docs": ["police_report.pdf", "photos.zip"],
        "expected_outcome": "APPROVE",
        "expected_payout": 2500
    },
    {
        "claim_id": "CLM-2024-002",
        "policy_number": "HOME-2024-PREMIUM",
        "claim_type": "home",
        "date_of_incident": "2024-02-05",
        "date_reported": "2024-02-06",
        "customer_name": "Sarah Johnson",
        "description": """Kitchen fire from stove. Fire department responded.
        Kitchen cabinets destroyed, ceiling damaged, smoke damage throughout.
        Temporary housing needed during repairs.""",
        "estimated_damage": 45000,
        "supporting_docs": ["fire_report.pdf", "damage_assessment.pdf"],
        "expected_outcome": "APPROVE",
        "expected_payout": 42500
    },
    {
        "claim_id": "CLM-2024-003",
        "policy_number": "HEALTH-2024-GOLD",
        "claim_type": "health",
        "date_of_incident": "2024-02-10",
        "date_reported": "2024-02-11",
        "customer_name": "Michael Chen",
        "description": """Emergency room visit for chest pain.
        Tests: EKG, chest X-ray, blood work, cardiac enzymes.
        Diagnosis: Anxiety attack (not cardiac). Released after 4 hours.
        In-network hospital.""",
        "total_charges": 5200,
        "supporting_docs": ["er_report.pdf", "itemized_bill.pdf"],
        "expected_outcome": "APPROVE"
    }
]

claims_df = pd.DataFrame(SAMPLE_CLAIMS)
print(f"✓ Created {len(SAMPLE_CLAIMS)} sample insurance claims")
print(f"\nClaims breakdown:")
print(claims_df['claim_type'].value_counts())
print(f"\nExpected outcomes:")
print(claims_df['expected_outcome'].value_counts())

---

## PART 2: RAG KNOWLEDGE BASE (20 minutes)

Building vector database for policy retrieval.

In [ ]:
# Chunk policy documents

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    separators=["\n\nSECTION", "\n\n", "\n", " "],
    length_function=len
)

all_chunks = []
for doc in policy_documents:
    chunks = text_splitter.split_documents([doc])
    all_chunks.extend(chunks)

print(f"✓ Created {len(all_chunks)} chunks from {len(policy_documents)} documents")
print(f"  Average chunk size: {np.mean([len(c.page_content) for c in all_chunks]):.0f} characters")

In [ ]:
# Initialize ChromaDB vector database

chroma_client = chromadb.Client(Settings(
    anonymized_telemetry=False,
    allow_reset=True
))

# Reset if exists
try:
    chroma_client.delete_collection("insurance_policies")
except:
    pass

# Create collection
insurance_collection = chroma_client.create_collection(
    name="insurance_policies",
    metadata={"description": "Insurance policy documents"}
)

print("✓ ChromaDB collection created")

In [ ]:
# Generate embeddings and index chunks

print("Generating embeddings and indexing...")

for i, chunk in enumerate(all_chunks):
    embedding = embeddings.embed_query(chunk.page_content)
    
    insurance_collection.add(
        ids=[f"chunk_{i}"],
        embeddings=[embedding],
        documents=[chunk.page_content],
        metadatas=[chunk.metadata]
    )
    
    if (i + 1) % 5 == 0:
        print(f"  Indexed {i+1}/{len(all_chunks)} chunks...")

print(f"\n✓ Indexed {len(all_chunks)} policy chunks")

In [ ]:
# Create policy retrieval function

def retrieve_policy_clauses(query: str, n_results: int = 3) -> List[Dict[str, Any]]:
    """Retrieve relevant policy clauses"""
    query_embedding = embeddings.embed_query(query)
    
    results = insurance_collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )
    
    retrieved = []
    for i, (doc, metadata, distance) in enumerate(zip(
        results['documents'][0],
        results['metadatas'][0],
        results['distances'][0]
    )):
        retrieved.append({
            'content': doc,
            'metadata': metadata,
            'similarity': 1 - distance,
            'rank': i + 1
        })
    
    return retrieved

# Test retrieval
print("Testing policy retrieval...")
test_results = retrieve_policy_clauses("auto collision coverage", n_results=2)
print(f"✓ Retrieved {len(test_results)} results")
print(f"  Top similarity: {test_results[0]['similarity']:.3f}")

---

## PART 3: CLAIMS PROCESSING AGENT (25 minutes)

Building agent with custom tools.

In [ ]:
# Fraud Detection Tool

def fraud_detection_tool_func(claim_data: str) -> str:
    """Analyze claim for fraud indicators"""
    try:
        data = json.loads(claim_data)
        red_flags = []
        fraud_score = 0
        
        # Check delayed reporting
        incident_date = datetime.fromisoformat(data.get('date_of_incident', '2024-01-01'))
        reported_date = datetime.fromisoformat(data.get('date_reported', '2024-01-01'))
        days_delay = (reported_date - incident_date).days
        
        if days_delay > 7:
            red_flags.append(f"Delayed reporting: {days_delay} days")
            fraud_score += 15
        
        if not data.get('supporting_docs'):
            red_flags.append("No supporting documentation")
            fraud_score += 20
        
        # Determine recommendation
        if fraud_score >= 40:
            recommendation = "MANUAL_REVIEW"
            risk_level = "HIGH"
        elif fraud_score >= 20:
            recommendation = "ENHANCED_VERIFICATION"
            risk_level = "MEDIUM"
        else:
            recommendation = "PROCEED"
            risk_level = "LOW"
        
        return json.dumps({
            "fraud_score": min(fraud_score, 100),
            "risk_level": risk_level,
            "red_flags": red_flags,
            "recommendation": recommendation
        }, indent=2)
    except Exception as e:
        return json.dumps({"error": str(e)})

fraud_detection_tool = Tool(
    name="fraud_detection",
    func=fraud_detection_tool_func,
    description="Analyzes claim for fraud. Input: JSON with dates and docs."
)

print("✓ Fraud detection tool created")

In [ ]:
# Coverage Verification Tool (uses RAG)

def coverage_verification_tool_func(verification_request: str) -> str:
    """Verify coverage using RAG"""
    try:
        data = json.loads(verification_request)
        policy_type = data.get('policy_type', 'auto')
        claim_description = data.get('claim_description', '')
        
        # Retrieve relevant clauses
        query = f"{policy_type} insurance coverage for {claim_description}"
        clauses = retrieve_policy_clauses(query, n_results=3)
        
        # Improved coverage determination
        coverage_text = " ".join([c['content'] for c in clauses]).lower()
        claim_desc_lower = claim_description.lower()
        
        # Check for specific exclusions that apply to this claim
        is_covered = True
        exclusion_found = None
        
        # Specific exclusion checks
        exclusion_keywords = {
            'intentional damage': 'intentional',
            'racing': 'racing',
            'commercial use': 'commercial',
            'flood': 'flood',
            'earthquake': 'earthquake',
            'cosmetic': 'cosmetic',
            'experimental': 'experimental'
        }
        
        for exclusion_type, keyword in exclusion_keywords.items():
            if keyword in claim_desc_lower:
                # Check if this is explicitly excluded
                if f"{keyword}" in coverage_text and ("exclusion" in coverage_text or "not covered" in coverage_text):
                    is_covered = False
                    exclusion_found = f"{exclusion_type.title()} is excluded by policy"
                    break
        
        # If no specific exclusion found, check for positive coverage indicators
        if is_covered:
            coverage_indicators = ['coverage', 'covers', 'covered', 'deductible', 'maximum coverage']
            has_coverage_info = any(indicator in coverage_text for indicator in coverage_indicators)
            
            # If we found coverage information and no explicit exclusion, mark as covered
            if has_coverage_info:
                is_covered = True
            else:
                # Default to covered for standard scenarios
                is_covered = True
        
        return json.dumps({
            "is_covered": is_covered,
            "relevant_clauses": [
                {"source": c['metadata']['source'], 
                 "content": c['content'][:200] + "...",
                 "similarity": c['similarity']}
                for c in clauses
            ],
            "exclusion_note": exclusion_found
        }, indent=2)
    except Exception as e:
        return json.dumps({"error": str(e)})

coverage_verification_tool = Tool(
    name="coverage_verification",
    func=coverage_verification_tool_func,
    description="Verifies coverage using policy docs. Input: JSON with policy_type and claim_description."
)

print("✓ Coverage verification tool created")

In [ ]:
# Payout Calculator Tool

def payout_calculator_tool_func(calculation_request: str) -> str:
    """Calculate insurance payout"""
    try:
        data = json.loads(calculation_request)
        
        total_damage = float(data.get('total_damage', 0))
        deductible = float(data.get('deductible', 0))
        coinsurance_pct = float(data.get('coinsurance', 100))
        coverage_limit = float(data.get('coverage_limit', float('inf')))
        
        # Calculate
        if total_damage <= deductible:
            payout = 0
            covered_amount = 0
            note = "Damage below deductible"
        else:
            after_deductible = total_damage - deductible
            covered_amount = after_deductible * (coinsurance_pct / 100)
            payout = min(covered_amount, coverage_limit)
            note = "Calculated within policy limits"
        
        return json.dumps({
            "total_damage": total_damage,
            "deductible": deductible,
            "covered_amount": round(covered_amount, 2),
            "payout": round(payout, 2),
            "calculation_note": note
        }, indent=2)
    except Exception as e:
        return json.dumps({"error": str(e)})

payout_calculator_tool = Tool(
    name="payout_calculator",
    func=payout_calculator_tool_func,
    description="Calculates payout. Input: JSON with total_damage, deductible, coinsurance."
)

print("✓ Payout calculator tool created")

---

## PART 4: ORCHESTRATION WORKFLOW (25 minutes)

Building complete claims processing pipeline.

In [ ]:
# Complete Orchestrated Claims Processing System

@dataclass
class ClaimProcessingResult:
    """Complete processing result"""
    claim_id: str
    status: str
    payout_amount: float
    processing_time: float
    fraud_score: int
    coverage_verified: bool
    policy_clauses_cited: List[str]
    reasoning: str
    audit_trail: List[Dict[str, Any]]
    timestamp: str


class ClaimsProcessingOrchestrator:
    """Orchestrated claims processing"""
    
    def __init__(self):
        self.processing_stats = {
            'total_processed': 0,
            'approved': 0,
            'denied': 0,
            'manual_review': 0
        }
    
    def process_claim(self, claim: Dict[str, Any]) -> ClaimProcessingResult:
        """Process claim through pipeline"""
        start_time = time.time()
        audit_trail = []
        claim_id = claim['claim_id']
        
        print(f"\n{'='*80}")
        print(f"PROCESSING CLAIM: {claim_id}")
        print(f"{'='*80}\n")
        
        # Step 1: Fraud Detection
        print("[1/5] Fraud Detection...")
        fraud_data = json.dumps({
            "date_of_incident": claim['date_of_incident'],
            "date_reported": claim['date_reported'],
            "estimated_damage": claim.get('estimated_damage', claim.get('total_charges', 0)),
            "supporting_docs": claim.get('supporting_docs', [])
        })
        
        fraud_result = fraud_detection_tool.func(fraud_data)
        fraud_analysis = json.loads(fraud_result)
        print(f"  Fraud Score: {fraud_analysis['fraud_score']}/100")
        
        audit_trail.append({"step": 1, "action": "fraud_detection"})
        
        if fraud_analysis['fraud_score'] >= 40:
            return ClaimProcessingResult(
                claim_id=claim_id,
                status="MANUAL_REVIEW",
                payout_amount=0,
                processing_time=time.time() - start_time,
                fraud_score=fraud_analysis['fraud_score'],
                coverage_verified=False,
                policy_clauses_cited=[],
                reasoning="High fraud risk - manual review",
                audit_trail=audit_trail,
                timestamp=datetime.now().isoformat()
            )
        
        # Step 2: Coverage Verification
        print("\n[2/5] Coverage Verification...")
        coverage_data = json.dumps({
            "policy_type": claim['claim_type'],
            "claim_description": claim['description']
        })
        
        coverage_result = coverage_verification_tool.func(coverage_data)
        coverage_analysis = json.loads(coverage_result)
        
        is_covered = coverage_analysis.get('is_covered', False)
        print(f"  Coverage: {'✓ COVERED' if is_covered else '✗ NOT COVERED'}")
        
        audit_trail.append({"step": 2, "action": "coverage_verification"})
        
        policy_clauses = [
            clause['source'] + ": " + clause['content'][:100]
            for clause in coverage_analysis.get('relevant_clauses', [])
        ]
        
        if not is_covered:
            return ClaimProcessingResult(
                claim_id=claim_id,
                status="DENIED",
                payout_amount=0,
                processing_time=time.time() - start_time,
                fraud_score=fraud_analysis['fraud_score'],
                coverage_verified=False,
                policy_clauses_cited=policy_clauses,
                reasoning="Claim denied - not covered",
                audit_trail=audit_trail,
                timestamp=datetime.now().isoformat()
            )
        
        # Step 3: Payout Calculation
        print("\n[3/5] Payout Calculation...")
        
        # Determine deductible
        if claim['claim_type'] == 'auto':
            deductible = 1000
            coinsurance = 100
        elif claim['claim_type'] == 'home':
            deductible = 2500
            coinsurance = 100
        else:  # health
            deductible = 2000
            coinsurance = 80
        
        payout_data = json.dumps({
            "total_damage": claim.get('estimated_damage', claim.get('total_charges', 0)),
            "deductible": deductible,
            "coinsurance": coinsurance,
            "coverage_limit": 50000
        })
        
        payout_result = payout_calculator_tool.func(payout_data)
        payout_analysis = json.loads(payout_result)
        payout_amount = payout_analysis['payout']
        
        print(f"  Payout: ${payout_amount:,.2f}")
        
        audit_trail.append({"step": 3, "action": "payout_calculation"})
        
        # Final decision
        status = "APPROVED"
        reasoning = f"""Claim APPROVED - Payout: ${payout_amount:,.2f}

Fraud: PASSED ({fraud_analysis['fraud_score']}/100)
Coverage: CONFIRMED
Payout: ${payout_analysis['total_damage']:,.2f} - ${payout_analysis['deductible']:,.2f} = ${payout_amount:,.2f}"""
        
        self.processing_stats['total_processed'] += 1
        self.processing_stats['approved'] += 1
        
        processing_time = time.time() - start_time
        
        print(f"\n{'='*80}")
        print(f"✓ {status} - ${payout_amount:,.2f}")
        print(f"  Time: {processing_time:.2f}s")
        print(f"{'='*80}\n")
        
        return ClaimProcessingResult(
            claim_id=claim_id,
            status=status,
            payout_amount=payout_amount,
            processing_time=processing_time,
            fraud_score=fraud_analysis['fraud_score'],
            coverage_verified=True,
            policy_clauses_cited=policy_clauses,
            reasoning=reasoning.strip(),
            audit_trail=audit_trail,
            timestamp=datetime.now().isoformat()
        )

orchestrator = ClaimsProcessingOrchestrator()
print("✓ Orchestrator initialized")

In [ ]:
# Process all sample claims

results = []

for claim in SAMPLE_CLAIMS:
    result = orchestrator.process_claim(claim)
    results.append(result)
    time.sleep(1)

# Statistics
print("\n" + "="*80)
print("STATISTICS")
print("="*80)
for key, value in orchestrator.processing_stats.items():
    print(f"  {key}: {value}")

---

## PART 5: RAGAS EVALUATION (20 minutes)

Evaluating system quality.

In [ ]:
# RAGAs Evaluation

if RAGAS_AVAILABLE and len(results) > 0:
    print("Preparing RAGAs evaluation...")
    
    eval_data = {
        "question": [],
        "answer": [],
        "contexts": [],
        "ground_truth": []
    }
    
    for result in results[:3]:
        eval_data["question"].append(f"Process claim {result.claim_id}")
        eval_data["answer"].append(result.reasoning)
        eval_data["contexts"].append(result.policy_clauses_cited)
        eval_data["ground_truth"].append(f"Claim should be {result.status}")
    
    ragas_dataset = Dataset.from_dict(eval_data)
    print(f"✓ Dataset with {len(ragas_dataset)} examples")
    
    print("\nRunning RAGAs evaluation...")
    
    try:
        ragas_results = evaluate(
            ragas_dataset,
            metrics=[faithfulness, answer_relevancy, context_precision, context_recall],
            llm=llm_gpt4,
            embeddings=embeddings
        )
        
        print("\n" + "="*80)
        print("RAGAS RESULTS")
        print("="*80)
        
        results_df = ragas_results.to_pandas()
        for metric in ['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall']:
            if metric in results_df.columns:
                score = results_df[metric].mean()
                print(f"  {metric.title()}: {score:.3f}")
        print("="*80)
    except Exception as e:
        print(f"⚠️ RAGAs error: {e}")
else:
    print("⚠️ RAGAs not available")

---

## PART 6: COMPLIANCE & PRODUCTION (15 minutes)

In [ ]:
# Compliance Reporting

class ComplianceReporter:
    @staticmethod
    def generate_audit_report(result: ClaimProcessingResult) -> str:
        report = f"""{'='*80}
INSURANCE CLAIM AUDIT REPORT
{'='*80}

CLAIM: {result.claim_id}
STATUS: {result.status}
PAYOUT: ${result.payout_amount:,.2f}
TIME: {result.processing_time:.2f}s

FRAUD: {result.fraud_score}/100
COVERAGE: {result.coverage_verified}

POLICY CLAUSES:
"""
        for clause in result.policy_clauses_cited:
            report += f"  - {clause}\n"
        
        report += f"\nREASONING:\n{result.reasoning}\n"
        report += f"{'='*80}"
        return report

if results:
    audit_report = ComplianceReporter.generate_audit_report(results[0])
    print(audit_report)

In [ ]:
# Production Monitoring

class ProductionMonitor:
    def __init__(self, results: List[ClaimProcessingResult]):
        self.results = results
    
    def get_dashboard(self) -> str:
        if not self.results:
            return "No claims processed"
        
        total = len(self.results)
        approved = sum(1 for r in self.results if r.status == 'APPROVED')
        total_payout = sum(r.payout_amount for r in self.results)
        avg_time = np.mean([r.processing_time for r in self.results])
        
        return f"""{'='*80}
PRODUCTION DASHBOARD
{'='*80}

CLAIMS: {total}
APPROVED: {approved} ({approved/total*100:.1f}%)

TOTAL PAYOUT: ${total_payout:,.2f}
AVG PAYOUT: ${total_payout/max(approved, 1):,.2f}

AVG TIME: {avg_time:.2f}s

STATUS: {'✓ HEALTHY' if avg_time < 60 else '⚠️ DEGRADED'}
{'='*80}"""

monitor = ProductionMonitor(results)
print(monitor.get_dashboard())